In [ ]:
import gradio as gr
import re
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿 설정
prompt_template = PromptTemplate(
    template="""경제전문가로서 다음 주어진 정보를 바탕으로 질문에 응답하시오. 1) 144자 정도의 완결된 문장으로 작성할 것 2) 개조식으로 작성하지 말 것 3) 한국은행이나 경제에 관련되지 않은 질문에 대한 답변은 거절할 것.

***Information***
{context}

***Question***
{query}

***Answer***""",
    input_variables=['context', 'query']
)

# 질문에 대한 답변 생성 함수 (RetrievalQA 활용)
def chat_with_retrieval(message, history):

    # 유사도가 높은 문장을 검색
    results = database.similarity_search(message, k=5) 

    #조회를 통해 얻은 정보를 저장
    context = "Relevant documents:\n"  

    for i, result in enumerate(results):
        context += f"""
        -----------------------------------
        Doc {i+1}:
        {result.page_content}
        """

    # 프롬프트 템플릿 적용
    prompt = prompt_template.format(context=context, query=message)
    
    # LLM을 통해 답변 생성
    llm_response = huggingface_llm(prompt)  # 여기서는 LLM 호출 결과가 문자열로 반환됩니다.
    result_text = llm_response  # 이미 문자열이므로 `get` 대신 바로 result_text로 사용

    # 불필요한 패턴 제거 로직
    if '***Answer***' in result_text: 
        answer_parsed = result_text.split('***Answer***')[-1].strip()
    elif '***information***' in result_text:
        answer_parsed = result_text.split('***Question***')[-1].strip()
    elif '***Answer***' in result_text:
        answer_parsed = result_text.split('***Answer***')[-1].strip()        
    else:
        answer_parsed = result_text.strip()

    # 불필요한 패턴 (공백 + 문자 + :) 제거
    answer_parsed = re.sub(r'\s\w+:', '', answer_parsed).strip()

    # 질문과 답변을 히스토리에 저장 (history는 대화 히스토리)
    history.append((message, answer_parsed))

    # Gradio가 (응답, history)를 반환해야 하므로, 대화 기록과 함께 반환
    return history, history
